# WEB scarping

İn this notebook we going to write a web scarping code for house rents scaring from<br>
<a href="https://www.zingat.com/" target=_blank>zingat.com</a>.  İn this part scarp only İstanbul data.<br>
After that we develop a regression model for prediction istanbul house rent prices and make a webisite.

<img src="https://i.imgur.com/6zM7JBq.png">

### Libraries

In [1]:
from undetected_chromedriver import Chrome
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import itertools
import pandas as pd

pd.set_option('display.max_columns', 80)

Firstly we will take all rent house adverts

In [2]:
# A function for scarp adverts
def take_adverts(url):
    # Use undetected_chromedriver as uc
    driver = Chrome()
    try:
        driver.get(url)
        # Wait for JavaScript to execute and load the content
        driver.implicitly_wait(10)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        links = soup.select('li.zl-card a.zl-card-inner')
        return list(map(lambda link: link.get('href'), links))
    except Exception as e:
        print(f"Hata oluştu: {e}")
    finally:
        # Quit the driver to release system resources
        driver.quit()

def get_all_adverts(base_url, total_pages):
    # Create all page URLs
    urls = map(lambda page: base_url.format(page), range(1, total_pages + 1))
    # Pull ads from all pages
    all_links_lists = map(take_adverts, urls)
    # Combine all ad links
    all_links = list(itertools.chain.from_iterable(all_links_lists))
    return all_links

Aplication to function

In [3]:
# Base URL and Total Pages Number
base_url = "https://www.zingat.com/istanbul-kiralik?page={}"
total_pages = 48

# Take all the ads
all_adverts = get_all_adverts(base_url, total_pages)

Secondly we will tek all data we want from all adverds

In [4]:
def extract_data(url):
    try:
        # Start the Chrome driver
        driver = Chrome()
        driver.get(url)
        
        # Wait for JavaScript to load
        driver.implicitly_wait(10)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Extract li elements with class 'col-md-6'
        li_elements = soup.find_all('li', class_='col-md-6')
        
        # Find the element with class "listing-price" and extract the text
        price_element = soup.find('strong', itemprop='price')
        price = price_element.text.strip() if price_element else "N/A"
        
        # Extract location data
        location_element = soup.find('h2')
        location = location_element.text.strip() if location_element else "N/A"
        
        # Extract information from each li element and add the price and location
        data = {li.find('strong').text.strip(): li.find('span').text.strip() for li in li_elements}
        data["Location"] = location
        data['Price'] = price
        
        # Create a DataFrame from the extracted data
        df = pd.DataFrame([data])
        return df
    
    except Exception as e:
        print(f"An error occurred while scraping {url}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame
    
    finally:
        # Close the driver
        driver.quit()

Usage of function

In [ ]:
dataframe=pd.DataFrame()
for url in all_adverts:
    try:
        scarp_url="https://www.zingat.com"+url
        df=extract_data(scarp_url)
        dataframe = pd.concat([dataframe, df], ignore_index=True)
    except:
        pass
try:
    dataframe=pd.DataFrame(dataframe)
    dataframe.shape
    dataframe.head()
except:
    pass

Save the data

In [14]:
dataframe.to_csv("rent_houses.csv", index=False)

Sources

-Zafer Acar Notes <br>-[https://chatgpt.com/]